# Miniproject 1 - Classification
## Classification, weight sharing, auxiliary losses

The objective of this project is to test different architectures to compare two digits visible in a two-channel image. It aims at showing in particular the impact of weight sharing, and of the use of an auxiliary loss to help the training of the main objective.

### 0. Import library and define python3 as default

In [5]:
#!/usr/bin/env python3
""" File to solve the first miniproject which is classification """

import torch
from torch import nn, optim
import dlc_practical_prologue as prologue


__author__ = 'Eugène Lemaitre, Natalie Bolón Brun, Louis Munier'
__version__ = '0.1'

### 1. Import data

In [6]:
def import_data(N, normalize):
    train_input, train_target, train_classes, \
    test_input, test_target, test_classes = prologue.generate_pair_sets(N)

    # Normalize data
    if normalize:
        mu, std = train_input.mean(), train_input.std()
        train_input.sub_(mu).div_(std)
        test_input.sub_(mu).div_(std)

    return train_input, train_target, test_input, test_target

### 2. Define the device to work on CUDA if it is available

In [7]:
def define_device(model, criterion, train_in, train_targ, test_in, test_targ):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    print(device)

    model.to(device)
    criterion.to(device)
    train_in, train_targ = train_in.to(device), train_targ.to(device)
    test_in, test_targ = test_in.to(device), test_targ.to(device)

    return train_in, train_targ, test_in, test_targ

### 3. Define model

In [8]:
class Net_Conv(nn.Module):
    def __init__(self, nb_hidden):
        super(Net_Conv, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=1))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

In [31]:
# Define variables
N = 1000
normalize = False

train_input, train_target, test_input, test_target = import_data(N, normalize)

train_input_1 = train_input[:,0,:,:]
train_input_2 = train_input[:,1,:,:]

print(train_input_1.size())

torch.Size([1000, 14, 14])
